### Simulation analysis

This notebook is used to compare the results of the VI-based TVP-BVAR to the MCMC-based TVP-BVAR. Note that the results of the MCMC-based TVP-BVAR are hardcoded into the code. This is due to the fact that these results are calculated in the results.R script. Make sure to select the correct simulation file for the VI-based TVP-BVAR and the correct results for the MCMC-based TVP-BVAR if you are going to compare the results.

In [187]:
with open(f'../simulations/results/statistics_{T}_{p}_{n_iterations}_{prior}_{sparsity}_huber_2.pkl', 'rb') as f:
        simulation_result = pickle.load(f)

In [188]:
# Select one of the results from the TVP for a comparison

# tvp_bar_result = np.array([1.525215, 1.568972, 1.569777, 1.576379, 1.572117, 1.573545, 1.563883, 1.560764]) # 100 0.20 M = 3
# tvp_bar_result = np.array([1.537108, 1.586357, 1.586235, 1.594027, 1.589423, 1.593814, 1.580771, 1.581765]) # 100 0.40 M = 3
# tvp_bar_result = np.array([2.103911, 2.127565, 2.154175, 2.144278, 2.142613, 2.138913, 2.155391, 2.164471]) # 200 0.20 M = 3
# tvp_bar_result = np.array([2.145268, 2.178971, 2.208494, 2.203912, 2.205018, 2.211773, 2.235132, 2.255868]) # 200 0.40 M = 3

# tvp_bar_result = np.array([2.431721, 2.530802, 2.554573, 2.570033, 2.582666, 2.570129, 2.583781, 2.603649]) # 100 0.20 M = 7
# tvp_bar_result = np.array([2.568019, 2.710974, 2.763319, 2.788549, 2.803765, 2.795588, 2.801986, 2.817411]) # 100 0.40 M = 7
# tvp_bar_result = np.array([3.930162, 4.023899, 4.096134, 4.123376, 4.122091, 4.132326, 4.121637, 4.145625]) # 200 0.20 M = 7
# tvp_bar_result = np.array([4.422962, 4.617997, 4.761793, 4.849296, 4.910502, 4.986810, 5.032367, 5.103598]) # 200 0.40 M = 7

In [189]:
simulation_result_block = np.block(simulation_result[0])

M_3 = simulation_result_block[:n_iterations]
M_7 = simulation_result_block[n_iterations:]

In [190]:
result_block = M_7

threshold_percentage = 10e-2

threshold_high = int(round(n_iterations - threshold_percentage*n_iterations,0))

msfe_list = result_block[:,:8].astype(np.float64).mean(1)
alpl_list = result_block[:,8:16].astype(np.float64).mean(1)

cleaned_indices = msfe_list.argsort()[:threshold_high]
msfe_list_cleaned = msfe_list[cleaned_indices]
alpl_list_cleaned = alpl_list[cleaned_indices]

msfe_h_step = result_block[cleaned_indices,:8].astype(np.float64).mean(0)

msd = result_block[cleaned_indices,16].astype(np.float64).mean()
alpl = alpl_list.mean()

In [191]:
msd = round(msd,6)
msfe = round(np.mean(msfe_list_cleaned),3)
alpl = round(alpl,2)

print(f"MSFE = {msfe}, MSD = {msd} & ALPL = {alpl}")

MSFE = 3.726, MSD = 0.013835 & ALPL = -29.52


In [192]:
msfe_ratio = np.round(msfe_h_step/tvp_bar_result,3)

print(f"VI/TVP = {msfe_ratio}")

VI/TVP = [0.678 0.826 0.856 0.836 0.803 0.764 0.723 0.68 ]


In [193]:
msfe_h_step.mean()/tvp_bar_result.mean()

0.7704240923688898

### Random walk (RW)

In [122]:
def calculate_rw_msfe(y, T, train, total_h=8):
    
    train_index = train
    number_of_predictions = T - train_index
    msfe_tvp = np.zeros(total_h)
    total_h = np.arange(1,total_h+1)

    for h in total_h:
            y_true_h = y[:,train_index:]
            y_pred_h = y[:,(train_index - h):-h]

            msfe_tvp[h-1] = np.mean((y_pred_h - y_true_h) ** 2)
            
    return msfe_tvp

In [123]:

train = T - 25
total_h = 8
n_iterations = 200
iterations_set = np.arange(1,n_iterations+1,1)
msfe_rw = np.zeros((n_iterations, total_h))

for run in iterations_set:

    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    msfe_rw[run-1] = calculate_rw_msfe(y, T, train, total_h)

In [124]:

threshold_percentage = 2.5e-2

threshold_high = int(round(n_iterations - threshold_percentage*n_iterations,0))

cleaned_indices = msfe_rw.mean(1).argsort()[:threshold_high]
msfe_rw_cleaned = msfe_rw[cleaned_indices]

h_step_rw = msfe_rw_cleaned.mean(0)

In [125]:
msfe_ratio = np.round(h_step_rw/tvp_bar_result,3)

print(f"RW/TVP = {msfe_ratio} \nMSFE: {round(h_step_rw.mean(),4)}")

RW/TVP = [1.358 1.747 1.91  2.016 2.055 2.067 2.072 2.111] 
MSFE: 4.1089


In [126]:
msfe_ratio = np.round(h_step_rw/msfe_h_step,3)

print(f"RW/VI = {msfe_ratio} \nMSFE: {round(h_step_rw.mean(),4)}")

RW/VI = [1.987 1.917 2.087 2.257 2.39  2.505 2.645 2.823] 
MSFE: 4.1089
